# Machine Translation

In diesem Notebook möchten wir uns mit der Königsdisziplin des Natural Language Processings beschäftigen, der maschinellen Übersetzung.
Vermutlich gibt es keine NLP-Anwendung, die einerseits vielen bekannt ist, anderseits durch Deep Learning und neuronalen Netzen einen solchen Aufschwung bekommen hat.
Vor dem Siegeszug der neuronalen Netzen, wurde maschinelle Übersetzung deshalb als schwierig angesehen, weil sie alle Teilaspekte von Sprache beinhaltet. Neben grammatikalischer Korrektheit, sollen maschinell übersetzte Texte den Sinn des Originaltextes wiedergeben und darüberhinaus auch den Subtext, wie Ironie, Witz, erfassen.
Ob letzteres einfach gelingt, sei hier mal dahingestellt.

## Daten
Im Vergleich zu den bisherigen Notebooks sind die Daten für machine translation etwas dröge. Daher möchten wir uns hier nicht allzu lange aufhalten. Speichert zunächst folgende Datei ab und entpackt sie: http://www.manythings.org/anki/fra-eng.zip

Danach wollen wir die Daten einlesen. Das Format ist einfach, pro Zeile steht ein Satzpaar getrennt von einem Tab.

In [0]:
from google.colab import drive

drive.mount('/data')

Drive already mounted at /data; to attempt to forcibly remount, call drive.mount("/data", force_remount=True).


In [0]:
with open('/data/My Drive/Colab Notebooks/data/fra.txt', 'r') as f:
    lines = f.readlines()

In [0]:
pairs = [tuple(l.split("\t")) for l in lines]
pairs = pairs[:5000]
# pairs

In [0]:
pairs_cleaned = [(e, f.strip()) for e, f, m in pairs]
pairs_cleaned

[('Go.', 'Va !'),
 ('Hi.', 'Salut !'),
 ('Hi.', 'Salut.'),
 ('Run!', 'Cours\u202f!'),
 ('Run!', 'Courez\u202f!'),
 ('Who?', 'Qui ?'),
 ('Wow!', 'Ça alors\u202f!'),
 ('Fire!', 'Au feu !'),
 ('Help!', "À l'aide\u202f!"),
 ('Jump.', 'Saute.'),
 ('Stop!', 'Ça suffit\u202f!'),
 ('Stop!', 'Stop\u202f!'),
 ('Stop!', 'Arrête-toi !'),
 ('Wait!', 'Attends !'),
 ('Wait!', 'Attendez !'),
 ('Go on.', 'Poursuis.'),
 ('Go on.', 'Continuez.'),
 ('Go on.', 'Poursuivez.'),
 ('Hello!', 'Bonjour !'),
 ('Hello!', 'Salut !'),
 ('I see.', 'Je comprends.'),
 ('I try.', "J'essaye."),
 ('I won!', "J'ai gagné !"),
 ('I won!', "Je l'ai emporté !"),
 ('I won.', 'J’ai gagné.'),
 ('Oh no!', 'Oh non !'),
 ('Attack!', 'Attaque !'),
 ('Attack!', 'Attaquez !'),
 ('Cheers!', 'Santé !'),
 ('Cheers!', 'À votre santé !'),
 ('Cheers!', 'Merci !'),
 ('Cheers!', 'Tchin-tchin !'),
 ('Get up.', 'Lève-toi.'),
 ('Go now.', 'Va, maintenant.'),
 ('Go now.', 'Allez-y maintenant.'),
 ('Go now.', 'Vas-y maintenant.'),
 ('Got it!', "J'a

Damit unser Modell weiß wo Anfang und Ende der französischen Sätze sind, müssen wir noch ein spezielles Start- (🏳️) und Endsymbol (🏴) einfügen:

In [0]:
pairs_cleaned_with_marker = [(e, "🏳️" + f + "🏴") for e, f in pairs_cleaned]
# pairs_cleaned_with_marker

Speichert euch nun alle Character, die in den englischen Sätzen und in den französischen Sätzen in jeweils einem Set ab.

In [0]:
english_characters = set(char for word in pairs_cleaned_with_marker for char in word[0])
french_characters = set(char for word in pairs_cleaned_with_marker for char in word[1])
# TODO
french_characters

{' ',
 '!',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f',
 '️',
 '🏳',
 '🏴'}

Um unser Modell später mit passenden Eingabedaten füttern zu können, müssen wir nun noch die jeweiligen Vektorlängen für den encoder und den decoder bestimmen. Speichert die Längen in den Variablen  `encoder_vector_len` und `decoder_vector_len` ab. Hinweis: uns Modell bekommt Vektoren, die die Characters pro Wort abbilden als Eingabe.

In [0]:
encoder_vector_len = len(english_characters) #TODO
decoder_vector_len = len(french_characters) #TODO
print(encoder_vector_len)
print(decoder_vector_len)

67
89


Um die Eingabevektoren bestimmen zu können, müssen wir uns noch Dictionaries bauen, die jeweils die Character auf die Komponenten des Vektors abbilden

In [0]:
english_token_index = {char: index for index, char in enumerate(english_characters)} #TODO 
french_token_index = {char: index for index, char in enumerate(french_characters)} #TODO
english_token_index

{' ': 0,
 '!': 31,
 '$': 35,
 '%': 54,
 '&': 13,
 "'": 42,
 ',': 52,
 '-': 29,
 '.': 55,
 '0': 37,
 '1': 50,
 '3': 15,
 '5': 59,
 '7': 30,
 '8': 64,
 '9': 25,
 ':': 28,
 '?': 5,
 'A': 62,
 'B': 26,
 'C': 23,
 'D': 14,
 'E': 10,
 'F': 36,
 'G': 56,
 'H': 44,
 'I': 45,
 'J': 32,
 'K': 58,
 'L': 66,
 'M': 16,
 'N': 2,
 'O': 48,
 'P': 46,
 'R': 33,
 'S': 61,
 'T': 51,
 'U': 43,
 'V': 38,
 'W': 49,
 'Y': 63,
 'a': 41,
 'b': 3,
 'c': 18,
 'd': 22,
 'e': 9,
 'f': 11,
 'g': 21,
 'h': 39,
 'i': 27,
 'j': 47,
 'k': 24,
 'l': 53,
 'm': 1,
 'n': 60,
 'o': 34,
 'p': 19,
 'q': 40,
 'r': 6,
 's': 57,
 't': 7,
 'u': 65,
 'v': 12,
 'w': 20,
 'x': 4,
 'y': 8,
 'z': 17}

Wir haben nun alle Dimensionen bestimmt, um die Trainingsdaten zu definieren. Dafür bauen wir uns drei Matrizen (eine als Eingabe für den Encoder, eine als Eingabe für den Decoder und eine als Ausgabe des Decoders) zusammen. Die Dimensionen dafür sind jeweils: Anzahl Sätze, Anzahl Wörter des längsten Satzes, Anzahl Characters.

In [0]:
import numpy as np

max_encoder_sentence_len = max((len(e) for e, _ in pairs_cleaned_with_marker))
max_decoder_sentence_len = max((len(f) for _, f in pairs_cleaned_with_marker))

print(max_encoder_sentence_len)
print(max_decoder_sentence_len)

print(len(pairs_cleaned_with_marker))
print(encoder_vector_len)
print(decoder_vector_len)


encoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_encoder_sentence_len, encoder_vector_len),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(pairs_cleaned_with_marker), max_decoder_sentence_len, decoder_vector_len),
    dtype='float32')

14
60
5000
67
89


Befüllt nun die Matrizen. Die `encoder_input_data` und die `decoder_input_data`-Matrix soll für jeden Satz aus den Trainingsdaten, einen Eintrag mit einem Token-Vektor pro Wort enthalten.
Die `decoder_target_data`-Matrix soll den gleichen Inhalt wie die `decoder_input_data`-Matrix haben, allerdings um eine Position verschoben. (Schaut euch nochmal die Folien an, wir brauchen Platz für das Startsymbol) 

Die `encoder_input_data`-Matrix wird mit den englischen Daten befüllt, die beiden anderen mit den französischen Daten.



In [0]:
for i, (e, f) in enumerate(pairs_cleaned_with_marker):
    for t, char in enumerate(e):
        encoder_input_data[i, t, english_token_index[char]] = 1.
    for t, char in enumerate(f):
        decoder_input_data[i, t, french_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t-1, french_token_index[char]] = 1.

## Sequence-to-sequence Modelle

Im Gegensatz zu den bisherigen Notebooks, wollen wir hier mit dem Modell beginnen. Einerseits, weil die Daten eher dröge sind (englisch-fränzosische Satzpaare), andererseits, weil das Modell selbst komplizierter ist und wir hier die Functional-API von Keras benutzen müssen.

Im Gegensatz zu der Sequence-API, definiert man hier ein Modell in dem, den jeweiligen Layer mit dem Vorgänger Layer aufruft:
```

a = Input(...)
b = Dense(...)

t = b(a)


```
Als Rückgabe bekommt man einen Tensor.


Ganz generell gesprochen, möchten wir hier ein Seq2Seq-Modell aufbauen, das auf LSTMs basiert. Wie auf den Folien beschrieben wurde, teilt sich so ein Modell in einen Encoder und Decoder auf. 
Wir beginnen die Implementierung mit dem Encoder.

### Encoder

Auch das komplexeste Modell fängt mit einer Eingabe ein.

Definiert einen Input-Layer, der als Eingabedimension, Sätze beliebiger Länge als Vektoren nimmt. Eine Komponente im Vektor repräsentiert einen Character. Unbekannte Teile der Eingabedimensionen kann man mit `None` definieren.

In [0]:
#model.load_weights("/data/My Drive/Colab Notebooks/data/model_machine_translation_5.h5")
#from keras.models import load_model
#del model
#model = load_model('/data/My Drive/Colab Notebooks/data/model_machine_translation_5.h5')
#model.summary()
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
#encoder_input = model.get_layer(name = 'input_1')
#decoder_input = model.get_layer(name = 'input_2')
#encoder_input

In [0]:
from keras.layers import Input

encoder_input = Input(shape=(None, encoder_vector_len))


Using TensorFlow backend.


Als nächstes definiert einen LSTM-Layer, der 256 hidden units hat. Die Ausgabe des Layers soll später an den Decoder 'verfüttert' werden. Seht in der Dokumentation nach wie man den Thought-Vektor zurückbekommen kann (https://keras.io/layers/recurrent/)

In [0]:
from keras.layers import LSTM

encoder = LSTM(units=256, return_state=True)

Verbindet nun den `encoder_input`-Layer mit dem `encoder`-Layer

In [0]:
_, state_h, state_c = encoder(encoder_input)

Der Encoder wäre damit fertig implementiert.

### Decoder

Auch der Decoder fängt mit einem Input-Layer an. Definiert einen Input-Layer analog zum Encoder.

In [0]:
decoder_input = Input(shape=(None, decoder_vector_len))

Wie auch im Encoder, ist das Herzstück ein LSTM-Layer. Definiert einen LSTM-Layer mit 256 hidden units. Seht in der Dokumentation nach, wie man neben den Thought-Vektoren, auch die komplette Ausgabe-Sequence zurückbekommt.

In [0]:
decoder_lstm = LSTM(256, return_state=True, return_sequences=True)

Verbindet nun den `decoder_input`-Layer mit dem `decoder_lstm`-Layer. An welcher der drei Ausgaben sind wir hier interessiert? Außerdem müssen wir hier, den Thought-Vektor des Encoders mit übergeben, dies geschieht über den `initial_state`-Parameter

In [0]:
decoder_outputs, _, _ = decoder_lstm(decoder_input, initial_state=[state_h, state_c])

Den Abschluss unseres Modells bildet ein Dense-Layer, der gleich viele `hidden_units` wie der Input-Layer des Decoders hat. Als Aktivierungsfunktion nehmen wir die Softmax-Funktion.

In [0]:
from keras.layers import Dense

decoder_layer = Dense(decoder_vector_len, activation="softmax")

Verbindet diesen Layer mit dem bisherigen Decoder.

In [0]:
decoder = decoder_layer(decoder_outputs)

Wir haben nun alle Elemente für ein Sequence-To-Sequence-Modell zusammen. Im letzten Schritt bauen wir das ganze Modell zusammen.

In [0]:
from keras import Model

model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 67)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 89)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 331776      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  354304      input_2[0][0]                    
                                                                 lstm_1[0][1]             

Fertig! Nun kann das Modell trainiert werden.

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 4000 samples, validate on 1000 samples
Epoch 1/100





4000/4000 [==============================] - 9s 2ms/step - loss: 0.9018 - val_loss: 0.9388
Epoch 2/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.8025 - val_loss: 0.8796
Epoch 3/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.7158 - val_loss: 0.7920
Epoch 4/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.6352 - val_loss: 0.7121
Epoch 5/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.5811 - val_loss: 0.6634
Epoch 6/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.5445 - val_loss: 0.6311
Epoch 7/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.5170 - val_loss: 0.6175
Epoch 8/100
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4933 - val_loss: 0.5819
Epoch 9/100
4000/4000 [===

In [0]:
#model.save("/data/My Drive/Colab Notebooks/data/model_machine_translation_5.h5")

### Inferenz

Um mit dem trainierten Modell Texte übersetzen zu können, müssen wir nun noch unser bisheriges Modell etwas umbauen.
Im ersten Schritt definieren wir uns ein `Model` das einen Eingabetext encodiert. Als Eingabe hat dieses `Model` den bisherigen `encoder_input`-Layer und die Thought-Vektoren als Output. 


In [0]:
encoder_model = Model(encoder_input, [state_h, state_c])


Etwas komplizierter ist das Decoder-Modell. Als Input nimmt es zum einen die Thought-Vektoren des Encoders, zum anderen den `decoder_input`-Layer von oben.
Definiert zunächst zwei Input-Layer für die Thought-Vektoren.

In [0]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))

Setzt diese beiden Input-Layer als `initial_state` und den `decoder_input`-Layer als Input in das `decoder_lstm` von oben ein.

In [0]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_input, initial_state=[decoder_state_input_h, decoder_state_input_c])

Setzt nun den `decoder_outputs`-Tensor in den `decoder_layer` von oben ein.

In [0]:
decoder_outputs = decoder_layer(decoder_outputs)

Nun können wir ein `Model` zusammenbauen, das uns als Decoder dient. Input ist hier der `decoder_input`-Layer, zusammen mit den `decoder_state_input_h` und dem `decoder_state_input_c`-Tensor. Als Output dient uns der `decoder_outputs`-Tensor und der `state_h` mit dem `state_c`-Tensor. 

In [0]:
decoder_model = Model(inputs=[decoder_input,decoder_state_input_h,decoder_state_input_c], outputs=[decoder_outputs, state_h, state_c])

Ein kleiner Zwischenschritt ist noch nötig, bevor wir mit dem Modell Texte übersetzen können. Damit wir aus unseren Vektoren wieder Texte bekommen, müssen wir noch das Dictionary `reverse_french_token_index` befüllen, das die "Umkehrung" von `french_token_index` ist.

In [0]:
reverse_french_token_index = {char: index for index, char in french_token_index.items()} #TODO
reverse_french_token_index
reverse_target_char_index = {char: index for index, char in english_token_index.items()} #TODO

Nun können wir eine Funktion `translate` implementieren, die als Eingabe einen englischen Text als Vektoren nimmt.

In [0]:
def translate(english_text):
    # Wandelt mit dem encoder_model, `english_text` in einen Thought-Vektor um
    thought = encoder_model.predict(english_text)
    
    # Als Input für den Decoder, wir zum einen der thought-Vektor benötigt, 
    # zum anderen die Vektor-Repräsentation des Startsymbols
    french_text = np.zeros((1, 1, decoder_vector_len))
    
    # Belegt die Komponente des Start-Symbols mit 1
    french_text[0, 0, french_token_index['🏳']] = 1.
    
    # und übergebt beides an den Decoder
    chars, h, c = decoder_model.predict([french_text] + thought)
    
    # chars beinhaltet eine 1 x 1 x decoder_vector_len Matrix, die für jeden französische Character
    # eine Art "Wahrscheinlichkeit" angibt.
    # Findet die Komponente, mit der größten Wahrscheinlichkeit und wandelt sie in ein Character um
    french_char_idx = np.argmax(chars[0, -1, :])
    
    french_char = reverse_french_token_index[french_char_idx]

    decoded_sentence = ''
    while french_char != '🏴':
      # die nächste Prediction wäre dann
      french_text = np.zeros((1, 1, decoder_vector_len))
      french_text[0, 0, french_char_idx] = 1.
      chars, h, c = decoder_model.predict([french_text, h, c])
      
      french_char_idx = np.argmax(chars[0, -1, :])
      
      french_char = reverse_french_token_index[french_char_idx]
      decoded_sentence += french_char
    
    # baut eine Schleife mit geeigneter Abbruchbedingung, die die Schritte von oben umfasst 
    # und dabei aus french_char einen übersetzen Text zusammenbaut
    return decoded_sentence[:-1]

In [0]:
for seq_index in range(400,415):
  english_text = encoder_input_data[seq_index:seq_index+1] 
  french_text = translate(english_text)
  print(pairs_cleaned[seq_index])
  print(french_text)


("Let's go!", 'Allons-y !')
Faisons-le !
("Let's go.", 'Allons-y !')
Allons-y !
('Look out!', 'Attention !')
Attentoir !
('Look out!', 'Regarde donc !')
Attentoir !
('Marry me.', 'Épouse-moi !')
Épousez-moi !
('Marry me.', 'Épousez-moi !')
Épousez-moi !
('May I go?', 'Puis-je partir ?')
Puis-je m'y rendre ?
('May I go?', 'Puis-je y aller ?')
Puis-je m'y rendre ?
('May I go?', "Puis-je m'y rendre ?")
Puis-je m'y rendre ?
('Save Tom.', 'Sauve Tom.')
Sauve Tom.
('Save Tom.', 'Sauvez Tom.')
Sauve Tom.
('She came.', 'Elle est venue.')
Elle est venue.
('She died.', 'Elle est morte.')
Elle est morte.
('She runs.', 'Elle court.')
Elle court.
('Sit down!', 'Assieds-toi !')
Assieds-toi !


### Hausaufgabe: 
 Implementiert die BLEU-Metrik wie auf den Folien nach und evaluiert euer Modell damit.

In [0]:
import math

def getMN(n, reference, translated_text):
  reference_tokens = reference.split()
  translation_tokens = translated_text.split()
  number_correct = 0
  for i in range(len(reference_tokens) - n + 1):
    n_gramm = ' '.join(reference_tokens[i:i+n])
    for j in range(len(translation_tokens) - n + 1):
      if n_gramm == ' '.join(translation_tokens[j:j+n]):
        number_correct += 1
  return number_correct

def getWN(n, translated_text):
  return len(translated_text.split()) - n + 1

def getMNRef(n, reference):
  return len(reference.split()) - n + 1

def getPN(n, reference, translated_text):
  return (min(getMN(n, reference, translated_text), getMNRef(n, reference))) / getWN(n, translated_text)

def getBLEU(reference, translated_text):
  bleu = 1
  lenTranslated = len(translated_text.split())
  lenReference = len(reference.split())
  r = min(5, min(lenReference, lenTranslated))
  for n in range(r):
    bleu *= getPN(n, reference, translated_text)
  return min(1, math.exp(1 - (lenReference/lenTranslated))) * (bleu ** (1/4))

In [0]:
getMN(3, "the cat sat on the mat", "the the on cat sat mat")
getPN(3, "the cat sat on the mat", "the cat sat on the mat")

1.0

In [0]:
n = 3
reference = "Israeli officials are responsible for airport security"
translated_text = "airport security Israeli officials are responsible"

print("getMN: ", getMN(n, reference, translated_text))
print("getWN: ", getWN(n, translated_text))
print("getMNRef: ", getMNRef(n, reference))
print("getPN: ", getPN(n, reference, translated_text))
print("getBLEU: ", getBLEU(reference, translated_text))

getMN:  2
getWN:  4
getMNRef:  5
getPN:  0.5
getBLEU:  0.5288716132517868


In [0]:
sumBleu = 0

for seq_index in range(len(pairs_cleaned)):
  english_text = encoder_input_data[seq_index:seq_index+1] 
  french_text = translate(english_text)
  sumBleu += getBLEU(pairs_cleaned[seq_index][1], french_text)

print(sumBleu / len(pairs_cleaned))

0.429910475420674
